# Team In The Cloud Final Project


Members: Anchita Bora, Hridya Antony, Nicole Carter


## Report Noisy Max Algorithm

Report Noisy Max (RNM) is a selection algorithm, which is an algorithm that finds the kth smallest value in a list. RNM is used to return the approximate maximizer and 2nd maximizer. According to the paper, RNM can also return the "noisy gap" between the approximate maximizer. 

Given a set of queries, RNM returns the identity (not value) of the query that is likely to have the largest value -- adds noise to each query answer and returns the index of the query with the largest noisy value. It accomplishes this by using a mechanism, an algorithm that supports differential privacy. In RNM according to the papers, Laplace Mechanism is used as it adds the _right_ amount of noise to satisfy differential privacy.

The basic idea:
1. For each case in a set, compute a noisy score using Laplace Mechanism
2. Output the element in the set with the maximum noisy score



In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

adult = pd.read_csv('adult_with_pii.csv')    

In [12]:
edu_options = adult['Education'].unique()
print(edu_options)
print( )

#Computes noisy score
def score(data, edu_option):
    return data.value_counts()[edu_option]/1000

noisy_score = score(adult['Education'], 'Bachelors')

print("Noisy score: " + str(noisy_score))

['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']

Noisy score: 5.355


#### Laplace Mechanism
Introduced earlier, Laplace Mechanism adds noise to achieve differential privacy. It does so by using the sensitivity of the function and epsilon for the amount of _noise_ added. In other words, the amount of required noise depends on the sensitivity of the query in question.  

According to the papers gathered, the mathematical definition of Laplace Mechanism is the following:

$$
F(x) = f(x) + Lap(s / (\epsilon))
$$


In [3]:
# sense = sensitivity 
# eps = epsilon aka amount of noise
# q = the query f(x)

def laplace_mechanism(q, sense, eps):
    return q + np.random.laplace(loc=0, scale=sense / eps)

In [11]:
def report_noisy_max(x, R, u, sense, eps):
    
    #Step 1: Computes scores for each element r in the set R while noisy_scores adds noise to each score.
    scores = [u(x,r) for r in R]
    
    noisy_scores = [laplace_mechanism(score, sense, eps) for score in scores]
    
    #Step 2: Finds index of the max score and returns element with that index.
    max_index = np.argmax(noisy_scores)
    
    return R[max_index]

#Shows the item/element with the largest noisy counts
noisiest_item = report_noisy_max(adult['Education'], edu_options, score, 1, 1)
print("Item (Education level) with Max Noise Score: \n" + noisiest_item + "\n\n")

#Lists items in order of most noise to least noise
print("Education levels from Most Noise to Least Noise:")
r = [report_noisy_max(adult['Education'], edu_options, score, 1, 1) for i in range(100)]
pd.Series(r).value_counts()
    

Item (Education level) with Max Noise Score: 
HS-grad


Education levels from Most Noise to Least Noise:


HS-grad         92
Some-college     5
Bachelors        3
dtype: int64

RNM satisfies differential privacy as it releases only the identity of the element with the largest noisy count and nothing more.